In [20]:
import sys, os
sys.path.append(os.path.realpath("/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages"))

import util_funcs
from importlib import reload
reload(util_funcs)
from copy import deepcopy as cp

import data_reader as read
import pandas as pd
import matplotlib.pyplot as plt
import pywt
import tsfresh.feature_extraction.feature_calculators as feats

import tsfresh

from os import path

reload(read)

edfRawData = read.EdfDataset("train", "01_tcp_ar", n_process=7, filter=False)

In [16]:
res = read.getAgesAndFileNames("train", "01_tcp_ar")
ages = [datum[1] for datum in res]
fn = [datum[0] for datum in res]
#associate first token file with each session for now
tokenFiles = []
for session_file in fn:
    session_dir = path.dirname(session_file)
    session_tkn_files = read.get_token_file_names(session_dir)
    session_tkn_files.sort()
    tokenFiles.append(session_tkn_files[0])
edfRawData.edf_tokens=tokenFiles

In [25]:
class HandEngineeredDataset(util_funcs.MultiProcessingDataset):
    # ndim = None
    # shape = None
    def __init__(self, edfFFTData, n_process=None, features = [], f_names = []):
        assert len(features) == len(f_names)
        self.edfFFTData = edfFFTData
        self.n_process = n_process
        if n_process is None:
            self.n_process = mp.cpu_count()
        self.features = features
        self.f_names = f_names
        # self.shape = np.asarray(self[0][0]).shape
        # self.ndim = len(self.shape)
        # self.shape = (len(self), *self.shape)

    def __len__(self):
        return len(self.edfFFTData)

    def __getitem__(self, i):
        if type(i) == slice:
            return self.getItemSlice(i)
        fftData, ann = self.edfFFTData[i]
        handEngineeredData = pd.DataFrame(index=fftData.columns, columns=self.f_names)
        
        for i, feature in enumerate(self.features):
            handEngineeredData[self.f_names[i]] = fftData.apply(lambda x: feature(x))
        return handEngineeredData

In [32]:
def norm_num_peaks_func(n):
    return lambda x: feats.number_peaks(x, n) / len(x)

def norm_num_vals_func(n):
    return lambda x: feats.number_peaks(-x, n) / len(x)

hed = HandEngineeredDataset(edfRawData, n_process = 7, features=[feats.abs_energy, max, min, norm_num_peaks_func(20), norm_num_vals_func(20)], f_names=["abs_energy", "max", "min", "num_peaks", "num_valleys"])

In [33]:
data = hed[0]

In [34]:
data

,abs_energy,max,min,num_peaks,num_valleys
EEG T5-REF,2.296780e+08,277.102893,-330.043849,0.020849,0.020046
EEG C3-REF,1.979039e+08,317.996415,-347.438855,0.020169,0.020058
EEG A1-REF,5.838565e+08,641.177315,-540.157210,0.020665,0.020677
EEG P4-REF,1.289074e+08,248.111216,-234.371317,0.020500,0.020273
EEG F8-REF,5.208032e+08,515.139816,-937.800942,0.018691,0.019016
EEG F3-REF,4.502753e+08,1013.338889,-559.535857,0.019016,0.019175
EEG CZ-REF,5.319490e+07,127.566877,-118.862374,0.019451,0.019126
EEG O2-REF,2.239007e+08,305.331630,-327.602445,0.020684,0.020512
EEG A2-REF,2.860068e+08,353.854541,-527.187250,0.020383,0.020561
EEG P3-REF,1.839775e+08,360.263228,-350.795786,0.020702,0.019917
